In [1]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.5/en_core_web_lg-2.2.5.tar.gz

     |████████████████████████████████| 827.9 MB 4.3 kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=f87ff7e75f7712be104d20cfce745dd16019c313d77ebdad384bbd64bafd1460
  Stored in directory: /root/.cache/pip/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg


In [1]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 1.7 MB/s 


In [2]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

'2020 slides'
 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'API key'
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
'Review Group 9.gdoc'
 Xizhi_Wu_UnderGrad_Transcript.pdf


In [4]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [5]:
!ls

 a0529-a0611_US		   Github
 a0529.csv		   heatmap529_UK.ipynb
 a0530.csv		   heatmap530.ipynb
 a0531.csv		   Heat_Map_Online_language.ipynb
 csv			   __MACOSX
 csv.zip		   non_violant1.csv
 dataframe_csv		   non_violant2.csv
 dataframe_csv_UK	   UK
 dataframe_result_csv	   violant.csv
 dataframe_result_csv_UK  '新建 Microsoft PowerPoint 演示文稿.pptx'


In [6]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [7]:
ct_path_531 = 'UK/06_01.csv'

In [8]:
ct = pd.read_csv(ct_path_531)

In [9]:
ct.shape[1]

47

In [10]:
#columns1=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message','expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media_type', 'media_numl', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']
columns2=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message', 'expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']

In [11]:
#len(columns1)
len(columns2)

47

In [12]:
ct.columns=columns2

In [13]:
ct

,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id
0,7528383899_10158705269673900,Facebook,2020-06-01 23:59:56,2021-08-31 03:22:28,link,"TODAY's Al Roker, Craig Melvin and Sheinelle J...",parade.com,The killing of George Floyd is spurring conver...,"Al Roker, Craig Melvin and Sheinelle Jones tal...",1.0,https://bit.ly/3gJP0Dt,https://www.facebook.com/7528383899/posts/1015...,173074,-3.833333,1.0,11,1,4,0,0,0,2,0,0,2,23,3,5,12,5,5,11,5,0,9,79220,PARADE Magazine,parademag,https://scontent-bos3-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,"More than just a magazine. ""Like"" PARADE for i...",True,2007-12-06 20:30:36,en,0,79220|10158705269673900
1,1090483797713057_3079536198807797,Facebook,2020-06-01 23:59:55,2021-06-24 03:05:51,photo,"TODAY's Al Roker, Craig Melvin and Sheinelle J...",parade.com,The killing of George Floyd is spurring conver...,A person working at the Woodbridge police depa...,2.0,https://www.facebook.com/photo.php?fbid=101579...,https://www.facebook.com/groups/10904837977130...,1413,-1.800000,1.0,4,2,15,0,4,0,0,0,0,0,5,3,7,3,4,10,9,4,0,2,10195422,"Dover Forum - Morris County, NJ",parademag,https://scontent-bos3-1.xx.fbcdn.net/v/t1.6435...,facebook_group,US,"More than just a magazine. ""Like"" PARADE for i...",False,2007-12-06 20:30:36,en,0,10195422|3079536198807797
2,134475425599_10158483699665600,Facebook,2020-06-01 23:59:55,2020-06-22 00:12:28,link,Statement | College Democrats of Notre Dame,sites.nd.edu,Statement FOR IMMEDIATE RELEASEContact: N’Kael...,You can read our full statement on the murder ...,2.0,http://sites.nd.edu/clubdems/home/statement,https://www.facebook.com/134475425599/posts/10...,1110,-0.818182,1.0,2,0,0,0,0,0,0,0,0,0,9,2,0,0,0,0,0,0,0,0,1373888,College Democrats of Notre Dame,NDCollegeDems,https://scontent-bos3-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,As the official chapter of College Democrats a...,False,2009-07-29 04:38:41,en,0,1373888|10158483699665600
3,207827199391299_1574941149346557,Facebook,2020-06-01 23:59:54,2021-07-14 11:54:39,photo,Statement | College Democrats of Notre Dame,sites.nd.edu,Statement FOR IMMEDIATE RELEASEContact: N’Kael...,Could be that too!!! But for sure!!! Somebody ...,1.0,http://sites.nd.edu/clubdems/home/statement,https://www.facebook.com/groups/20782719939129...,1050,-0.928571,1.0,0,0,0,1,0,0,0,0,0,0,3,2,2,2,1,3,1,0,0,1,8520454,Grass Roots Walkers,NDCollegeDems,https://scontent-bos3-1.xx.fbcdn.net/v/t1.1816...,facebook_group,US,As the official chapter of College Democrats a...,False,2009-07-29 04:38:41,en,0,8520454|1574941149346557
4,213488875757629_982576702182172,Facebook,2020-06-01 23:59:54,2021-07-07 07:31:29,photo,Statement | College Democrats of Notre Dame,sites.nd.edu,Statement FOR IMMEDIATE RELEASEContact: N’Kael...,LIVERPOOL PLAYERS SHOW SUPPORT TO THE 'BLACK L...,1.0,https://www.facebook.com/photo.php?fbid=116026...,https://www.facebook.com/groups/21348887575762...,27104,-117.000000,1.0,2,0,0,0,0,0,0,0,0,0,155,12,29,8,4,11,13,2,0,2,10142259,Fiji Samachar,NDCollegeDems,https://scontent-bos3-1.xx.fbcdn.net/v/t1.6435...,facebook_group,US,As the official chapter of College Democrats a...,False,2009-07-29 04:38:41,en,0,10142259|982576702182172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [14]:
ct['message'] = ct['message'].fillna('')
ct['description'] = ct['description'].fillna('')
ct['title'] = ct['title'].fillna('')

In [15]:
nlp = spacy.load('en_core_web_lg', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [16]:
def ner(txt):
    ner = dict()

    doc = nlp(txt)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [17]:
def ner_inline(doc):
    ner = dict()

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [18]:
def ner_pipe(texts):
    batch = []
    for doc in nlp.pipe(texts, batch_size=200, n_threads=16):
        batch.append(ner_inline(doc))
    return batch

In [19]:
ct['NER-msg'] = ner_pipe(ct['message'])

In [20]:
ct['NER-msg'] = ct['message'].apply(ner)

In [21]:
ct['NER-desc'] = ct['description'].apply(ner)

In [22]:
ct['NER-title'] = ct['title'].apply(ner)

In [23]:
ct[['NER-msg', 'NER-desc']].head()

,NER-msg,NER-desc
0,{},{}
1,{},{}
2,{},{}
3,{},{}
4,"{'Liverpool': 1, 'Minneapolis': 1, 'US': 1}",{}


In [24]:
ct.to_csv('a0601.csv')

In [25]:
ct=pd.read_csv('a0601.csv')

In [26]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [27]:
ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [28]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
4,"{'Liverpool': 1, 'Minneapolis': 1, 'US': 1}",{},{}
5,{'Beaumont': 1},{},{}
8,{'Newark': 1},{'Newark': 1},{'Newark': 1}
9,"{'America': 1, 'San Diego': 1}",{'Newark': 1},{'Newark': 1}
11,"{'U.S.': 3, 'America': 2, 'Minneapolis': 1}",{'United States': 1},{}


In [29]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [30]:
ct_ner['NER-msg']

4                            [Minneapolis, Liverpool, US]
5                                              [Beaumont]
8                                                [Newark]
9                                    [America, San Diego]
11                           [Minneapolis, America, U.S.]
                               ...                       
55621                                                  []
55622                                                  []
55623    [Minneapolis, the City of Minneapolis, Hilliard]
55624                                                  []
55626                                                  []
Name: NER-msg, Length: 39228, dtype: object

In [31]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
ct_ner['NER']

4                             [Minneapolis, Liverpool, US]
5                                               [Beaumont]
8                                 [Newark, Newark, Newark]
9                     [America, San Diego, Newark, Newark]
11             [Minneapolis, America, U.S., United States]
                               ...                        
55621                                           [Michigan]
55622                                           [Michigan]
55623    [Minneapolis, the City of Minneapolis, Hilliar...
55624                                           [NEW YORK]
55626                                        [ABILENE, TX]
Name: NER, Length: 39228, dtype: object

In [33]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [34]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [35]:
ner_lst = ct_ner['NER'].values

In [36]:
ner_lst

array([list(['minneapolis', 'liverpool', 'us']), list(['beaumont']),
       list(['newark']), ...,
       list(['minneapolis', 'the city of minneapolis', 'michigan', 'hilliard']),
       list(['new york']), list(['abilene', 'tx'])], dtype=object)

In [37]:
m=0
list=[]
for n in ner_lst:
  for i in range(len(n)):
    list.append(ner_lst[m][i])
  m+=1

In [38]:
list

['minneapolis',
 'liverpool',
 'us',
 'beaumont',
 'newark',
 'america',
 'san diego',
 'newark',
 'minneapolis',
 'united states',
 'america',
 'u.s.',
 'missouri',
 'urbana',
 'edinburg',
 'edinburg',
 'texas',
 'troy',
 'troy',
 'america',
 'troy',
 'troy',
 'minneapolis',
 'california',
 'minneapolis',
 'us',
 'united states',
 'us',
 'd.c.',
 'us',
 'seattle',
 'seattle',
 'ohio',
 'ohio',
 'dc',
 'us',
 'atlanta',
 'baltimore',
 'minnesota',
 'baltimore',
 'minnesota',
 'america',
 'coral',
 'miami-dade chie',
 'the u. s.',
 'minneapolis',
 'minneapolis',
 'bend',
 'coral',
 'decatur',
 'mlk',
 'minneapolis',
 'minneapolis',
 'des moines police',
 'minneapolis',
 'minneapolis',
 'chandler',
 'minneapolis',
 'chandler',
 'u.s.',
 'minnesota',
 'philando castile',
 'minnesota',
 'minneapolis',
 'america',
 'minneapolis',
 'america',
 'mayfield',
 'minneapolis',
 'america',
 'south carolina',
 'minnesota',
 'america',
 'charleston',
 'minneapolis',
 'oakland',
 'california',
 'd.c.'

In [39]:
c = collections.Counter()

In [40]:
dict_a=collections.Counter(list).most_common()

In [41]:
dict_a

[('minneapolis', 14569),
 ('america', 4833),
 ('us', 4498),
 ('minnesota', 4308),
 ('u.s.', 3672),
 ('the united states', 3471),
 ('washington', 1112),
 ('chicago', 1018),
 ('new york', 871),
 ('los angeles', 823),
 ('atlanta', 787),
 ('london', 732),
 ('california', 726),
 ('flint', 721),
 ('d.c.', 668),
 ('new york city', 600),
 ('usa', 576),
 ('michigan', 562),
 ('camden', 550),
 ('seattle', 548),
 ('houston', 520),
 ('texas', 501),
 ('boston', 497),
 ('nyc', 458),
 ('florida', 440),
 ('uk', 429),
 ('philadelphia', 396),
 ('oakland', 377),
 ('hennepin county', 375),
 ('dc', 368),
 ('china', 360),
 ('louisville', 338),
 ('canada', 335),
 ('the united states of america', 335),
 ('georgia', 332),
 ('miami', 317),
 ('brooklyn', 312),
 ('new zealand', 281),
 ('berlin', 281),
 ('russia', 278),
 ('denver', 276),
 ('germany', 270),
 ('sacramento', 268),
 ('australia', 263),
 ('portland', 256),
 ('nj', 252),
 ('dallas', 252),
 ('los angeles county', 248),
 ('united states', 235),
 ('kentucky

In [42]:
dataframe531=pd.DataFrame(dict_a)

In [43]:
len(dict_a)

5007

In [44]:
dataframe531.to_csv('dataframe0601_UK.csv')